In [1]:
import requests
import json

In [2]:
DEFAULT_API_URL = "https://agents-course-unit4-scoring.hf.space"
api_url = DEFAULT_API_URL
questions_url = f"{api_url}/questions"
submit_url = f"{api_url}/submit"


In [3]:
# Fetch Questions
def fetch_questions():
    """
    Fetch questions from the API.
    Returns a tuple of (error_message, questions_data).
    """
    # Check if the API URL is valid
    if not api_url.startswith("http"):
        return "Invalid API URL", None

    # Fetch questions from the API
    print(f"API URL: {api_url}")
    print(f"Fetching questions from: {questions_url}")
    try:
        response = requests.get(questions_url, timeout=15)
        response.raise_for_status()
        questions_data = response.json()
        if not questions_data:
                print("Fetched questions list is empty.")
                return "Fetched questions list is empty or invalid format.", None
        print(f"Fetched {len(questions_data)} questions.")
        return questions_data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching questions: {e}")
        return f"Error fetching questions: {e}", None
    except requests.exceptions.JSONDecodeError as e:
            print(f"Error decoding JSON response from questions endpoint: {e}")
            print(f"Response text: {response.text[:500]}")
            return f"Error decoding server response for questions: {e}", None
    except Exception as e:
        print(f"An unexpected error occurred fetching questions: {e}")
        return f"An unexpected error occurred fetching questions: {e}", None

In [4]:
questions = fetch_questions()

API URL: https://agents-course-unit4-scoring.hf.space
Fetching questions from: https://agents-course-unit4-scoring.hf.space/questions
Fetched 20 questions.


In [5]:
for question in questions:
    print(f"Task ID: {question['task_id']}")
    print(f"Question: {question['question']}")
    print(f"Level: {question['Level']}")
    print(f"File Name: {question['file_name']}")
    print("-" * 40 + "\n")

Task ID: 8e867cd7-cff9-4e6c-867a-ff5ddc2550be
Question: How many studio albums were published by Mercedes Sosa between 2000 and 2009 (included)? You can use the latest 2022 version of english wikipedia.
Level: 1
File Name: 
----------------------------------------

Task ID: a1e91b78-d3d8-4675-bb8d-62741b4b68a6
Question: In the video https://www.youtube.com/watch?v=L1vXCYZAYYM, what is the highest number of bird species to be on camera simultaneously?
Level: 1
File Name: 
----------------------------------------

Task ID: 2d83110e-a098-4ebb-9987-066c06fa42d0
Question: .rewsna eht sa "tfel" drow eht fo etisoppo eht etirw ,ecnetnes siht dnatsrednu uoy fI
Level: 1
File Name: 
----------------------------------------

Task ID: cca530fc-4052-43b2-b130-b30968d8aa44
Question: Review the chess position provided in the image. It is black's turn. Provide the correct next move for black which guarantees a win. Please provide your response in algebraic notation.
Level: 1
File Name: cca530fc-4052-43

In [6]:
def get_question_file(question):
    """
    Get the file associated with a question if it exists.
    
    Args:
        question (dict): The question dictionary containing task_id and file_name
    
    Returns:
        tuple: (file_content, file_path) if file exists, (None, None) otherwise
    """
    if not question.get("file_name"):
        print(f"No file associated with question {question['task_id']}")
        return None, None
    
    file_name = question["file_name"]
    task_id = question["task_id"]
    
    # Construct file URL using the correct endpoint format
    file_url = f"{api_url}/files/{task_id}"
    
    print(f"Fetching file from: {file_url}")
    try:
        response = requests.get(file_url, timeout=15)
        response.raise_for_status()
        
        # Create a directory to store downloaded files if it doesn't exist
        import os
        os.makedirs("downloaded_files", exist_ok=True)
        
        # Save the file locally
        local_path = f"./downloaded_files/{file_name}"
        with open(local_path, "wb") as f:
            f.write(response.content)
        print(f"Saved file to {local_path}")
        return response.content, local_path
            
    except requests.exceptions.RequestException as e:
        print(f"Error fetching file {file_name}: {e}")
        return None, None

In [7]:
for question in questions:
    if question['file_name']:
        print(f"Task ID: {question['task_id']}")
        print(f"Question: {question['question']}")
        print(f"File Name: {question['file_name']}")

        get_question_file(question)

        print("-" * 40 + "\n")

        

Task ID: cca530fc-4052-43b2-b130-b30968d8aa44
Question: Review the chess position provided in the image. It is black's turn. Provide the correct next move for black which guarantees a win. Please provide your response in algebraic notation.
File Name: cca530fc-4052-43b2-b130-b30968d8aa44.png
Fetching file from: https://agents-course-unit4-scoring.hf.space/files/cca530fc-4052-43b2-b130-b30968d8aa44
Saved file to ./downloaded_files/cca530fc-4052-43b2-b130-b30968d8aa44.png
----------------------------------------

Task ID: 99c9cc74-fdc8-46c6-8f8d-3ce2d3bfeea3
Question: Hi, I'm making a pie but I could use some help with my shopping list. I have everything I need for the crust, but I'm not sure about the filling. I got the recipe from my friend Aditi, but she left it as a voice memo and the speaker on my phone is buzzing so I can't quite make out what she's saying. Could you please listen to the recipe and list all of the ingredients that my friend described? I only want the ingredients fo